This notebook reads in photometry data in the format of CSV files from The Open Supernova Catalog, and CSV files output by my DOLPHOT-scraping script dolphot_retrieval_finalized.py and converts it to text files for input into Matt Nicholl's superbol.py program (code and documentation found here https://github.com/mnicholl/superbol).

The code takes as input the data from both sources, and the luminosity distance lum_d in Mpc (labelled d_L on the sne.space page for each SN). It subtracts the distance modulus (5log10(lum_d(10^6)/10)) and foreground galactic extinction (which must be looked up and added into the code by the user) from each magnitude value,  and converts the data into text files of the format 'MJD    filter1_mag    filter1_uncertainty    filter2_mag    filter2_uncertainty ... '.  The text files are named in the format 'SNname_filters.txt', for example 'SN2006aj_ugriz.txt' or 'iPTF13bvn_JHK.txt'.

It also collects the sources of each sne.space data set used for referencing purposes.

The code contains a function to perform this analysis and formatting specific to each SN, so the user will need to edit the code for use with data from other SNe.

In [1]:
#import dependencies
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import csv
from astropy.time import Time

In [ ]:
#read in data from The Open Supernova Catalog (input correct filenames or paths)
iPTF13bvn = pd.read_csv('sne_space_photometry_data/iPTF13bvn_photometry.txt')
SN2006aj = pd.read_csv('sne_space_photometry_data/SN2006aj_photometry.txt')
SN2007uy = pd.read_csv('sne_space_photometry_data/SN2007uy_photometry.txt')
SN1993J = pd.read_csv('sne_space_photometry_data/SN1993J_photometry.txt')

#read in data scraped from dolphot (input correct filenames or paths)
iPTF13bvn_lt = pd.read_csv('phot_plot_data_iPTF13bvn_old.csv')
SN1993J_lt = pd.read_csv('phot_plot_data_SN1993J_old.csv')
SN2007uy_lt = pd.read_csv('phot_plot_data_2007uy.csv')
SN2006aj_lt = pd.read_csv('phot_plot_data_2006aj.csv')

The next two cells define and then evaluate a function to analyze and format the data from both sources into two separate text files for one specific SN.

In [25]:
#define a function to format both data sets for iPTF13bvn
def format_for_superbol_iPTF13bvn(data, data_lt, lum_d):
    
    srcs = []
    
    #remove unneccesary column from sne.space data
    data.drop(['upperlimit'], axis=1)
    
    #remove data points with no associated error from sne.space data
    data = data[data.e_magnitude > 0]
    
    #select desired filters from sne.space data
    data = data[data.band.isin(['B', 'V', 'UVW2','I'])]
    
    #round datetime stamp to the day
    data.time = np.round(data.time, 0)
    
    #convert dates from dolphot data to MJD
    newcol = []    
    for i in data_lt.date_obs:
        newcol.append(np.round(Time(i+'T00:00:00.000').mjd,0))
    data_lt['time'] = newcol
    
    #calculate distance modulus and print (will need as input for superbol)
    dmod = (5*np.log10(lum_d*(10**6)/10.))
    print(dmod)
    
    #list sources of data taken from sne.space, for referencing purposes
    for x in data['source']:
        if x not in srcs:
            srcs.append(x)
            
    #format sne.space data and write to a text file
    file1 = open('iPTF13bvn_UBVI.txt', 'w+')      
     
    #group by observation date
    for k in data.time.unique():
        j = data[data.time == k]

        #select the data in this filter and average the magnitudes and magnitude uncertainties
        #for each observation date 
        b = j[j.band == 'B']
        B = np.average(b['magnitude'])
        B_err= np.average(b['e_magnitude'])
        #subtract the foreground galactic extinction for this filter and host galaxy, 
        #and the distance modulus for this object
        B = B - 0.184 - dmod

        #follow the same process for the other filters        
        v = j[j.band == 'V']
        V = np.average(v['magnitude'])
        V_err = np.average(v['e_magnitude'])
        V = V - 0.139 - dmod    

        u = j[j.band == 'UVW2']
        U = np.average(u['magnitude'])
        U_err = np.average(u['e_magnitude'])
        U = U - 0.220 - dmod

        i = j[j.band == 'I']
        I = np.average(i['magnitude'])
        I_err = np.average(i['e_magnitude'])

        #write to a tab delimited text file
        file1.writelines([str(k)+'\t', str(U)+'\t', str(U_err)+'\t', str(B)+'\t', str(B_err)+'\t', str(V)+'\t', str(V_err)+'\t', str(I)+'\t', str(I_err)+'\t \n'])    
    
    file1.close()
     
    #follow the same process as above to format dolphot data and write to a separate text file
    file2 = open('iPTF13bvnlt_UBVI.txt', 'w+')  
        
    for k in data_lt.time.unique():
        j = data_lt[data_lt.time == k]

        b = j[j.band.isin(['Extremely wide blue','WFPC2 B'])]
        B = np.average(b['m_avg'])
        B = B - 0.173 - dmod
        B_err= np.average(b['sigma_m_avg'])

        v = j[j.band == 'WFPC2 V']
        V = np.average(v['m_avg'])
        V = V - 0.145 - dmod
        V_err = np.average(v['sigma_m_avg'])    

        u = j[j.band == 'UV wide']
        U = np.average(u['m_avg'])
        U = U - 0.355 - dmod
        U_err = np.average(u['sigma_m_avg'])

        i = j[j.band == 'Wfpc 2 Wide I']
        I = np.average(i['m_avg'])
        I = I - 0.078 - dmod
        I_err = np.average(i['sigma_m_avg'])  

        file2.writelines([str(k)+'\t', str(U)+'\t', str(U_err)+'\t', str(B)+'\t', str(B_err)+'\t', str(V)+'\t', str(V_err)+'\t', str(I)+'\t', str(I_err)+'\t \n'])    
    

    file2.close()
    
    #print references
    return srcs  

In [26]:
#evaluate function to create text files for iPTF13bvn sne.space and dolphot photometry data
#(input names of data files read in at beginning of notebook, and luminosity distance from 
#sne.space) and print reference codes
format_for_superbol_iPTF13bvn(iPTF13bvn, iPTF13bvn_lt, 19.94)

31.498625769878185


['2016A&A...593A..68F', '2016ApJ...825L..22F', '2014Ap&SS.354...89B']

The cells below follow a similar process for a few more SNe.

In [10]:
def format_for_superbol_1993J(data, data_lt, lum_d):
    
    srcs = []
    
    data.drop(['upperlimit','instrument','telescope'], axis=1)
    #instead of removing data with no associated uncertainty, assign a value of 0.1 to the uncertainty for that data
    data = data.fillna(0.1)
    
    data = data[data.band.isin(['U','B', 'V', 'R','I'])]
    data.time = np.round(data.time, 0)
    
    newcol = []    
    for i in data_lt.date_obs:
        newcol.append(np.round(Time(i+'T00:00:00.000').mjd,0))
    data_lt['time'] = newcol
    
    dmod = (5*np.log10(lum_d*(10**6)/10.))
    print(dmod)
    
    for x in data['source']:
        if x not in srcs:
            srcs.append(x)

    file1 = open('SN1993J_UBVRI.txt', 'w+')      
        
    for k in data.time.unique():
        j = data[data.time == k]

        u = j[j.band == 'U']
        U = np.average(u['magnitude'])
        U = U - 0.348 - dmod
        U_err = np.average(u['e_magnitude'])

        b = j[j.band == 'B']
        B = np.average(b['magnitude'])
        B = B - 0.291 - dmod
        B_err= np.average(b['e_magnitude'])

        v = j[j.band == 'V']
        V = np.average(v['magnitude'])
        V = V - 0.220 - dmod
        V_err = np.average(v['e_magnitude'])    

        r = j[j.band == 'R']
        R = np.average(r['magnitude'])
        R = R - 0.174 - dmod
        R_err = np.average(r['e_magnitude'])

        i = j[j.band == 'I']
        I = np.average(i['magnitude'])
        I = I - 0.121 - dmod
        I_err = np.average(i['e_magnitude'])  

        file1.writelines([str(k)+'\t', str(U)+'\t', str(U_err)+'\t', str(B)+'\t', str(B_err)+'\t', str(V)+'\t', str(V_err)+'\t', str(R)+'\t', str(R_err)+'\t', str(I)+'\t', str(I_err)+'\t \n'])    
    
    file1.close()
        
    file2 = open('SN1993Jlt_UBVRI.txt', 'w+')  
        
    for k in data_lt.time.unique():
        j = data_lt[data_lt.time == k]
        
        u = j[j.band == 'ISM feature']
        U = np.average(u['m_avg'])
        U = U - 0.622 - dmod
        U_err = np.average(u['sigma_m_avg'])
        
        b = j[j.band == 'WFPC2 B']
        B = np.average(b['m_avg'])
        B = B - 0.291 - dmod
        B_err= np.average(b['sigma_m_avg'])

        v = j[j.band == 'WFPC2 V']
        V = np.average(v['m_avg'])
        V = V - 0.229 - dmod
        V_err = np.average(v['sigma_m_avg'])    

        r = j[j.band == 'SDSS r']
        R = np.average(r['m_avg'])
        R = R - 0.181 - dmod
        R_err = np.average(r['sigma_m_avg'])

        i = j[j.band == 'WFPC2 wide I']
        I = np.average(i['m_avg'])
        I = I - 0.124 - dmod
        I_err = np.average(i['sigma_m_avg'])  

        file2.writelines([str(k)+'\t', str(U)+'\t', str(U_err)+'\t', str(B)+'\t', str(B_err)+'\t', str(V)+'\t', str(V_err)+'\t', str(R)+'\t', str(R_err)+'\t',str(I)+'\t', str(I_err)+'\t \n'])    
    

    file2.close()
    
    return srcs  

In [11]:
format_for_superbol_1993J(SN1993J,SN1993J_lt,2.9)

27.311989989494784


/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['1995A&AS..110..513B,SUSPECT',
 '1996AJ....112..732R,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1994AJ....107.1022R,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1994supe.conf...27L,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1994AJ....107.1022R,1994supe.conf...27L,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1995A&AS..110..513B,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1994AJ....107.1453B,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1993PASJ...45L..59V,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1993PASJ...45L..63O,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1995PAZh...21..670M,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1994supe.conf...27L,1994AJ....107.1453B,Sternberg Astronomical Institute Supernova Light Curve Catalogue',
 '1994AJ....107.1022R,1994AJ....107.14

In [12]:
def format_for_superbol_2007uy(data, data_lt, lum_d):
    
    srcs = []
    
    data.drop(['upperlimit'], axis=1)
    data = data[data.e_magnitude > 0]
    
    data = data[data.band.isin(['V',"i'"])]
    data.time = np.round(data.time, 0)
    
    newcol = []    
    for i in data_lt.date_obs:
        newcol.append(np.round(Time(i+'T00:00:00.000').mjd,0))
    data_lt['time'] = newcol
    
    dmod = (5*np.log10(lum_d*(10**6)/10.))
    print(dmod)
    
    for x in data['source']:
        if x not in srcs:
            srcs.append(x)

    file1 = open('SN2007uy_VI.txt', 'w+')      
        
    for k in data.time.unique():
        j = data[data.time == k]

        v = j[j.band == 'V']
        V = np.average(v['magnitude'])
        V = V - 0.062 - dmod
        V_err = np.average(v['e_magnitude'])    

        i = j[j.band == "i'"]
        I = np.average(i['magnitude'])
        I = I - 0.038 - dmod
        I_err = np.average(i['e_magnitude'])  

        file1.writelines([str(k)+'\t', str(V)+'\t', str(V_err)+'\t', str(I)+'\t', str(I_err)+'\t \n'])    
    
    file1.close()
        
    file2 = open('SN2007uylt_VI.txt', 'w+')  
        
    for k in data_lt.time.unique():
        j = data_lt[data_lt.time == k]

        v = j[j.band == 'WFPC2 V']
        V = np.average(v['m_avg'])
        V = V - 0.064 - dmod
        V_err = np.average(v['sigma_m_avg'])    

        i = j[j.band == 'WFPC2 wide I']
        I = np.average(i['m_avg'])
        I = I - 0.035 - dmod
        I_err = np.average(i['sigma_m_avg'])  

        file2.writelines([str(k)+'\t', str(V)+'\t', str(V_err)+'\t', str(I)+'\t', str(I_err)+'\t \n'])    
    

    file2.close()
    
    return srcs  

In [13]:
format_for_superbol_2007uy(SN2007uy,SN2007uy_lt,26)

32.07486673985409


['2014ApJS..213...19B', '2014Ap&SS.354...89B']

In [14]:
def format_for_superbol_2006aj(data, data_lt, lum_d):
    
    srcs = []
    
    data.drop(['upperlimit'], axis=1)
    data = data[data.e_magnitude > 0]
    
    data = data[data.band.isin(['B', 'V', "r'","i'"])]
    data.time = np.round(data.time, 0)
    
    newcol = []   
    for i in data_lt.date_obs:
        newcol.append(np.round(Time(i+'T00:00:00.000').mjd,0))
    data_lt['time'] = newcol
    
    dmod = (5*np.log10(lum_d*(10**6)/10.))
    print(dmod)
    
    for x in data['source']:
        if x not in srcs:
            srcs.append(x)

    file1 = open('SN2006aj_BVRI.txt', 'w+')      
        
    for k in data.time.unique():
        j = data[data.time == k]

        b = j[j.band == 'B']
        B = np.average(b['magnitude'])
        B = B - 0.527 - dmod
        B_err= np.average(b['e_magnitude'])

        v = j[j.band == 'V']
        V = np.average(v['magnitude'])
        V = V - 0.399 - dmod
        V_err = np.average(v['e_magnitude'])    

        r = j[j.band == "r'"]
        R = np.average(r['magnitude'])
        R = R - 0.332 - dmod
        R_err = np.average(r['e_magnitude'])

        i = j[j.band == "i'"]
        I = np.average(i['magnitude'])
        I = I - 0.247 - dmod
        I_err = np.average(i['e_magnitude'])  

        file1.writelines([str(k)+'\t', str(B)+'\t', str(B_err)+'\t', str(V)+'\t', str(V_err)+'\t', str(R)+'\t', str(R_err)+'\t', str(I)+'\t', str(I_err)+'\t \n'])    
    
    file1.close()
        
    file2 = open('SN2006ajlt_BVRI.txt', 'w+')  
        
    for k in data_lt.time.unique():
        j = data_lt[data_lt.time == k]
        
        b = j[j.band == 'Johnson B']
        B = np.average(b['m_avg'])
        B = B - 0.527 - dmod
        B_err= np.average(b['sigma_m_avg'])

        v = j[j.band == 'WFPC2 V']
        V = np.average(v['m_avg'])
        V = V - 0.399 - dmod
        V_err = np.average(v['sigma_m_avg'])    

        r = j[j.band == "SDSS r'"]
        R = np.average(r['m_avg'])
        R = R - 0.332 - dmod
        R_err = np.average(r['sigma_m_avg'])

        i = j[j.band == 'Wfpc 2 Wide I']
        I = np.average(i['m_avg'])
        I = I - 0.219 - dmod
        I_err = np.average(i['sigma_m_avg'])  

        file2.writelines([str(k)+'\t', str(B)+'\t', str(B_err)+'\t', str(V)+'\t', str(V_err)+'\t', str(R)+'\t', str(R_err)+'\t', str(I)+'\t', str(I_err)+'\t \n'])    
    

    file2.close()
    
    return srcs  

In [15]:
format_for_superbol_2006aj(SN2006aj,SN2006aj_lt,149.8)

35.87755906681724


['2014Ap&SS.354...89B',
 '2014ApJS..213...19B',
 '2006ApJ...645L..21M,CfA Supernova Archive']